# 14_physical_variables_x_betas 
#### (Betas v1, because v2 were never used)

In [19]:
a="""
TA_Miramar perception
    variable    beta
0    den_pop  0.6619
1  den_inter  0.6587
2   land_use  0.5325
3   ind_ndvi  0.4381
4  grade_abs  0.7167
5   sidewalk  0.3690

TA_Providencia perception
    variable    beta
0    den_pop  0.6310
1  den_inter  0.7937
2   land_use  0.5595
3   ind_ndvi  0.8433
4  grade_abs  0.9008
5   sidewalk  0.6329

TA_Colinas perception
    variable    beta
0    den_pop  0.6750
1  den_inter  0.7417
2   land_use  0.6167
3   ind_ndvi  0.6250
4  grade_abs  0.7250
5   sidewalk  0.4625
"""

## __Import libraries__

In [1]:
first_folder_path = '../../../'

In [2]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import random
import scipy.stats as stats

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## __Load physical variables and select by polygon__

### Load physical variables

In [3]:
edges_var_gdl = gpd.read_file(first_folder_path + f"data/processed/volvo/edges_var_gdl/edges_var_gdl.gpkg")

# Show
print(edges_var_gdl.crs)
print(edges_var_gdl.shape)
edges_var_gdl.head(2)

EPSG:32613
(197439, 20)


,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,banqueta_pct,pobtot_u,pobtot_v,total_pobtot,average_population,dens_u,dens_v,average_density,interdens_u,interdens_v,average_intersdens,geometry
0,67637870229114485,67640019229114922,0,0.0,0.118923,1.144632,179.791185,69.176583,0.722146,80.0,27.0,107.0,53.5,153.566318,111.104363,132.335341,19.498764,19.071526,19.285145,"MULTILINESTRING ((676378.709 2291144.858, 6763..."
1,67640019229114922,67640526229117504,0,0.0,0.188798,0.317370,181.014513,68.547498,0.725329,27.0,5.0,32.0,16.0,111.104363,31.101049,71.102706,19.071526,19.041632,19.056579,"MULTILINESTRING ((676400.196 2291149.223, 6764..."


### Load polygons

In [5]:
polys = gpd.read_file(first_folder_path + f"data/processed/volvo/polys_gdl/Poligonos.gpkg")

# Show
print(polys.crs)
print(polys.shape)
polys

EPSG:32613
(3, 2)


,name,geometry
0,miramar,"POLYGON ((659404.786 2283009.827, 660250.980 2..."
1,providencia,"POLYGON ((668619.092 2288755.457, 668816.754 2..."
2,colinas,"POLYGON ((665060.447 2289080.643, 665057.510 2..."


In [14]:
edges_var_gdl.columns

Index(['u', 'v', 'key', 'grade_abs', 'ndvi_mean', 'entropy',
       'sidewalk_available', 'no_sidewalk_available', 'banqueta_pct',
       'pobtot_u', 'pobtot_v', 'total_pobtot', 'average_population', 'dens_u',
       'dens_v', 'average_density', 'interdens_u', 'interdens_v',
       'average_intersdens', 'geometry'],
      dtype='object')

## __Miramar__

In [16]:
# Select Miramar's edges
edges_var_miramar = edges_var_gdl.sjoin(polys.loc[polys.name=='miramar'])
edges_var_miramar.drop(columns='index_right',inplace=True)

# Add Miramar's betas
edges_var_miramar['beta_den_pop'] = 0.6619
edges_var_miramar['beta_den_inter'] = 0.6587
edges_var_miramar['beta_land_use'] = 0.5325
edges_var_miramar['beta_ind_ndvi'] = 0.4381
edges_var_miramar['beta_grade_abs'] = 0.7167
edges_var_miramar['beta_sidewalk'] = 0.3690

# Multiply Miramar's physical variables by Miramar's betas
edges_var_miramar['WI_den_pop'] = edges_var_miramar['average_density']*edges_var_miramar['beta_den_pop']
edges_var_miramar['WI_den_inter'] = edges_var_miramar['average_intersdens']*edges_var_miramar['beta_den_inter']
edges_var_miramar['WI_land_use'] = edges_var_miramar['entropy']*edges_var_miramar['beta_land_use']
edges_var_miramar['WI_ind_ndvi'] = edges_var_miramar['ndvi_mean']*edges_var_miramar['beta_ind_ndvi']
edges_var_miramar['WI_grade_abs'] = edges_var_miramar['grade_abs']*edges_var_miramar['beta_grade_abs']
edges_var_miramar['WI_sidewalk'] = edges_var_miramar['sidewalk_available']*edges_var_miramar['beta_sidewalk']

# Show
print(edges_var_miramar.crs)
print(edges_var_miramar.shape)
edges_var_miramar.head(2)

EPSG:32613
(453, 33)


,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,banqueta_pct,pobtot_u,...,beta_land_use,beta_ind_ndvi,beta_grade_abs,beta_sidewalk,WI_den_pop,WI_den_inter,WI_land_use,WI_ind_ndvi,WI_grade_abs,WI_sidewalk
61332,66039321228374187,66043571228373413,0,0.023178,0.058935,1.628174,245.659360,34.239189,0.877673,82.0,...,0.5325,0.4381,0.7167,0.369,104.634726,22.173172,0.867003,0.025819,0.016612,90.648304
61333,66042318228368862,66043571228373413,0,0.000000,0.050877,1.928047,220.854125,89.161779,0.712396,87.0,...,0.5325,0.4381,0.7167,0.369,102.676673,21.695744,1.026685,0.022289,0.000000,81.495172


## __Providencia__

In [17]:
# Select Providencia's edges
edges_var_providencia = edges_var_gdl.sjoin(polys.loc[polys.name=='providencia'])
edges_var_providencia.drop(columns='index_right',inplace=True)

# Add Providencia's betas
edges_var_providencia['beta_den_pop'] = 0.6310
edges_var_providencia['beta_den_inter'] = 0.7937
edges_var_providencia['beta_land_use'] = 0.5595
edges_var_providencia['beta_ind_ndvi'] = 0.8433
edges_var_providencia['beta_grade_abs'] = 0.9008
edges_var_providencia['beta_sidewalk'] = 0.6329

# Multiply Providencia's physical variables by Providencia's betas
edges_var_providencia['WI_den_pop'] = edges_var_providencia['average_density']*edges_var_providencia['beta_den_pop']
edges_var_providencia['WI_den_inter'] = edges_var_providencia['average_intersdens']*edges_var_providencia['beta_den_inter']
edges_var_providencia['WI_land_use'] = edges_var_providencia['entropy']*edges_var_providencia['beta_land_use']
edges_var_providencia['WI_ind_ndvi'] = edges_var_providencia['ndvi_mean']*edges_var_providencia['beta_ind_ndvi']
edges_var_providencia['WI_grade_abs'] = edges_var_providencia['grade_abs']*edges_var_providencia['beta_grade_abs']
edges_var_providencia['WI_sidewalk'] = edges_var_providencia['sidewalk_available']*edges_var_providencia['beta_sidewalk']

# Show
print(edges_var_providencia.crs)
print(edges_var_providencia.shape)
edges_var_providencia.head(2)

EPSG:32613
(187, 33)


,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,banqueta_pct,pobtot_u,...,beta_land_use,beta_ind_ndvi,beta_grade_abs,beta_sidewalk,WI_den_pop,WI_den_inter,WI_land_use,WI_ind_ndvi,WI_grade_abs,WI_sidewalk
13801,66843389228953530,66826254228956520,0,0.011513,0.209893,0.965143,330.065236,0.0,1.0,33.0,...,0.5595,0.8433,0.9008,0.6329,18.939662,11.806724,0.539997,0.177003,0.010371,208.898288
13802,66843389228953530,66843502228961764,0,0.048363,0.165909,1.374894,328.824381,0.0,1.0,33.0,...,0.5595,0.8433,0.9008,0.6329,22.900259,12.090116,0.769253,0.139911,0.043566,208.112951


## __Colinas__

In [18]:
# Select Colinas's edges
edges_var_colinas = edges_var_gdl.sjoin(polys.loc[polys.name=='colinas'])
edges_var_colinas.drop(columns='index_right',inplace=True)

# Add Colinas's betas
edges_var_colinas['beta_den_pop'] = 0.6750
edges_var_colinas['beta_den_inter'] = 0.7417
edges_var_colinas['beta_land_use'] = 0.6167
edges_var_colinas['beta_ind_ndvi'] = 0.6250
edges_var_colinas['beta_grade_abs'] = 0.7250
edges_var_colinas['beta_sidewalk'] = 0.4625

# Multiply Colinas's physical variables by Colinas's betas
edges_var_colinas['WI_den_pop'] = edges_var_colinas['average_density']*edges_var_colinas['beta_den_pop']
edges_var_colinas['WI_den_inter'] = edges_var_colinas['average_intersdens']*edges_var_colinas['beta_den_inter']
edges_var_colinas['WI_land_use'] = edges_var_colinas['entropy']*edges_var_colinas['beta_land_use']
edges_var_colinas['WI_ind_ndvi'] = edges_var_colinas['ndvi_mean']*edges_var_colinas['beta_ind_ndvi']
edges_var_colinas['WI_grade_abs'] = edges_var_colinas['grade_abs']*edges_var_colinas['beta_grade_abs']
edges_var_colinas['WI_sidewalk'] = edges_var_colinas['sidewalk_available']*edges_var_colinas['beta_sidewalk']

# Show
print(edges_var_colinas.crs)
print(edges_var_colinas.shape)
edges_var_colinas.head(2)

EPSG:32613
(141, 33)


,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,banqueta_pct,pobtot_u,...,beta_land_use,beta_ind_ndvi,beta_grade_abs,beta_sidewalk,WI_den_pop,WI_den_inter,WI_land_use,WI_ind_ndvi,WI_grade_abs,WI_sidewalk
984,66506542228883037,66508023228882465,0,0.000000,0.089006,0.000000,63.017331,104.677485,0.375786,0.0,...,0.6167,0.625,0.725,0.4625,6.006881,8.367833,0.000000,0.055629,0.000000,29.145515
986,66505104228924061,66513831228924264,0,0.034417,0.276073,0.679421,271.746961,45.118772,0.857609,17.0,...,0.6167,0.625,0.725,0.4625,42.966720,7.777926,0.418999,0.172546,0.024952,125.682969


## __Join all polygons__

In [21]:
edges_var_betas_polys = pd.concat([edges_var_miramar,edges_var_providencia])
edges_var_betas_polys = pd.concat([edges_var_betas_polys,edges_var_colinas])

# Show
print(edges_var_betas_polys.crs)
print(edges_var_betas_polys.shape)
edges_var_betas_polys.head(2)

EPSG:32613
(781, 33)


,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,banqueta_pct,pobtot_u,...,beta_land_use,beta_ind_ndvi,beta_grade_abs,beta_sidewalk,WI_den_pop,WI_den_inter,WI_land_use,WI_ind_ndvi,WI_grade_abs,WI_sidewalk
61332,66039321228374187,66043571228373413,0,0.023178,0.058935,1.628174,245.659360,34.239189,0.877673,82.0,...,0.5325,0.4381,0.7167,0.369,104.634726,22.173172,0.867003,0.025819,0.016612,90.648304
61333,66042318228368862,66043571228373413,0,0.000000,0.050877,1.928047,220.854125,89.161779,0.712396,87.0,...,0.5325,0.4381,0.7167,0.369,102.676673,21.695744,1.026685,0.022289,0.000000,81.495172


## __Save__

In [25]:
edges_var_betas_polys.to_file(first_folder_path + f"data/processed/volvo/edges_var_betas_polys/edges_var_betas_polys.gpkg")